In [28]:
import os


print("Sebelum:", os.getcwd())
os.chdir('/home/lenov/Documents/warung')
print("Sesudah:", os.getcwd())

from agents.mcp import MCPServerStdio
from agents import Agent, Runner, trace
from IPython.display import Markdown, display

import gradio as gr

Sebelum: /home/lenov/Documents/warung
Sesudah: /home/lenov/Documents/warung


In [38]:
import sys
from pathlib import Path

ROOT_DIR = Path().resolve().parent / "warung/app/mcp_sample"
print(ROOT_DIR)
sys.path.append(str(ROOT_DIR))


from app.mcp_sample.produk_database import init_db as init_db_produk, create_product_in_db
from app.mcp_sample.transaction_database import init_db as init_db_transaction

init_db_produk()
init_db_transaction()

sample_product_data_list = [
    {
        "nama_barang": "Indomie Goreng",
        "harga": 3000,
        "lokasi": "Rak Mie Instan",
        "deskripsi_suara_lokasi": "Ada di rak tengah, bagian mie instan.",
        "path_qris": "/qris/indomie_goreng.png",
        "stok": 50
    },
    {
        "nama_barang": "Aqua Botol 600ml",
        "harga": 3500,
        "lokasi": "Kulkas Minuman",
        "deskripsi_suara_lokasi": "Di dalam kulkas minuman, sebelah kanan.",
        "path_qris": "/qris/aqua_600ml.png",
        "stok": 100
    },
    {
        "nama_barang": "Chitato Sapi Panggang",
        "harga": 10000,
        "lokasi": "Rak Snack",
        "deskripsi_suara_lokasi": "Rak snack, di bagian atas.",
        "path_qris": "/qris/chitato_sapi_panggang.png",
        "stok": 30
    },
    {
        "nama_barang": "Susu Ultra Coklat 250ml",
        "harga": 6000,
        "lokasi": "Kulkas Susu",
        "deskripsi_suara_lokasi": "Kulkas khusus produk susu, di baris kedua dari atas.",
        "path_qris": "/qris/susu_ultra_coklat.png",
        "stok": 45
    },
    {
        "nama_barang": "Roti Tawar Sari Roti",
        "harga": 15000,
        "lokasi": "Rak Roti",
        "deskripsi_suara_lokasi": "Area roti, dekat kasir.",
        "path_qris": "/qris/sari_roti_tawar.png",
        "stok": 20
    },
    {
        "nama_barang": "Teh Botol Sosro Kotak",
        "harga": 4000,
        "lokasi": "Kulkas Minuman",
        "deskripsi_suara_lokasi": "Kulkas minuman, di sebelah kiri bawah.",
        "path_qris": "/qris/teh_botol_kotak.png",
        "stok": 70
    },
    {
        "nama_barang": "Sabun Mandi Lifebuoy",
        "harga": 5000,
        "lokasi": "Rak Perlengkapan Mandi",
        "deskripsi_suara_lokasi": "Bagian sabun dan sampo, rak nomor tiga.",
        "path_qris": "/qris/lifebuoy_sabun.png",
        "stok": 60
    },
    {
        "nama_barang": "Pasta Gigi Pepsodent",
        "harga": 8500,
        "lokasi": "Rak Perlengkapan Mandi",
        "deskripsi_suara_lokasi": "Di rak perlengkapan mandi, dekat dengan sikat gigi.",
        "path_qris": "/qris/pepsodent.png",
        "stok": 35
    },
    {
        "nama_barang": "Minyak Goreng Bimoli 1L",
        "harga": 25000,
        "lokasi": "Rak Minyak & Sembako",
        "deskripsi_suara_lokasi": "Rak sembako, di bagian minyak goreng.",
        "path_qris": "/qris/bimoli_1l.png",
        "stok": 25
    },
    {
        "nama_barang": "Kopi Kapal Api Special Mix",
        "harga": 12000, # Harga per renceng atau box kecil
        "lokasi": "Rak Kopi & Teh",
        "deskripsi_suara_lokasi": "Area kopi, di rak paling atas.",
        "path_qris": "/qris/kapal_api_special_mix.png",
        "stok": 40
    }
]

def reset_db():
    init_db_produk()
    init_db_transaction()

    for product_data in sample_product_data_list:
        created_id = create_product_in_db(product_data)
        # print(f"Produk {product_data['nama_barang']} ditambahkan dengan ID: {created_id}")

2025-06-06 14:26:18,642 - produk_database - INFO - init_db success - dropped and recreated table
2025-06-06 14:26:18,645 - transaction_database - INFO - init_db called


2025-06-06 14:26:18,672 - transaction_database - INFO - init_db finished


/home/lenov/Documents/warung/app/mcp_sample


In [127]:
from langchain_ollama import ChatOllama
from langchain.prompts import ChatPromptTemplate

llm = ChatOllama(
    model="llama3-groq-tool-use:latest",  # Specify the model (e.g., llama3.1, llama3.1:70b)
    base_url="http://localhost:11434",  # Default Ollama server URL
    temperature=0.1,
    top_p=0.5,
    seed=1337
)


prompt = ChatPromptTemplate.from_template("Answer this: {question}")
chain = prompt | llm
# response = chain.invoke({"question": "What is the capital of France?"})
# print(response.content)

In [129]:
from typing import TypedDict, List, Optional
from langchain.memory import ConversationBufferMemory
from produk import ProdukService, Produk

def get_memory():
    return ConversationBufferMemory(return_messages=True)

reset_db()

import asyncio
import gradio as gr
from langgraph.graph import StateGraph, END
from typing import TypedDict, List, Optional, Dict, Any
import datetime
import json
import re
from dataclasses import dataclass
from agents.mcp import MCPServerStdio
from langchain_core.messages import HumanMessage

class ChatState(TypedDict):
    messages: List[str]
    current_message: str
    chat_history: List[Dict[str, str]]
    produk_terpilih: Optional[List[Dict]]
    transaksi: Optional[Dict]
    detail_transaksi: List[Dict]
    produk_list: List[Dict]
    context: Dict[str, Any]
    intent: Optional[str]  # 'chat', 'produk', 'transaksi'
    response: str

class WarungAIChatbot:
    def __init__(self):
        self.params_produk = {"command": "uv", "args": ["run", "app/mcp_sample/produk_server.py"]}
        self.params_transaction = {"command": "uv", "args": ["run", "app/mcp_sample/transaction_server.py"]}
        self.chat_history = []
        self.current_state = None
        
        # Initialize graph
        self.graph = self._build_graph()
    
    def _build_graph(self):
        """Build the LangGraph workflow"""
        builder = StateGraph(ChatState)
        
        # Add nodes
        builder.add_node("chat_handler", self._chat_handler_node)
        builder.add_node("produk_handler", self._produk_handler_node)
        builder.add_node("transaksi_handler", self._transaksi_handler_node)
        builder.add_node("response_generator", self._response_generator_node)
        
        # Set entry point
        builder.set_entry_point("chat_handler")
        
        # Add conditional edges
        builder.add_conditional_edges(
            "chat_handler",
            self._route_from_chat,
            {
                "produk": "produk_handler",
                "transaksi": "transaksi_handler", 
                "response": "response_generator"
            }
        )
        
        builder.add_edge("produk_handler", "response_generator")
        builder.add_edge("transaksi_handler", "response_generator")
        builder.add_edge("response_generator", END)
        
        return builder.compile()
    
    async def _chat_handler_node(self, state: ChatState) -> ChatState:
        """Main chat handler that analyzes intent and manages conversation"""
        message = state['current_message'].lower()
        
        # Analyze intent
        intent = await self._analyze_intent(state['current_message'])
        state['intent'] = intent
        
        # Try to answer from current state/context first
        if intent == 'chat':
            response = await self._generate_chat_response(state)
            if response:
                state['response'] = response
                return state
        
        # If can't answer from state, determine routing
        state['context'] = state.get('context', {})
        return state
    
    async def _analyze_intent(self, message: str) -> str:
        """Analyze user intent using LLM"""
        prompt = f"""
        Analisis pesan user berikut dan tentukan intent utamanya:
        
        Pesan: "{message}"
        
        Intent yang tersedia:
        1. "produk" - jika user bertanya tentang produk, mencari produk, ingin beli sesuatu, tanya harga, stok
        2. "transaksi" - jika user ingin buat transaksi, bayar, checkout, lihat transaksi, update/hapus transaksi
        3. "chat" - jika user ngobrol biasa, salam, terima kasih, pertanyaan umum
        
        Jawab hanya dengan salah satu: produk, transaksi, atau chat
        """
        
        # Simulate LLM call - replace with actual LLM
        try:
            # response = await llm.ainvoke([HumanMessage(content=prompt)])
            # For now, use simple keyword matching
            message_lower = message.lower()
            if any(word in message_lower for word in ['beli', 'produk', 'harga', 'stok', 'mau', 'cari', 'ada']):
                return 'produk'
            elif any(word in message_lower for word in ['transaksi', 'bayar', 'checkout', 'buat', 'hapus', 'update']):
                return 'transaksi'
            else:
                return 'chat'
        except:
            return 'chat'
    
    async def _generate_chat_response(self, state: ChatState) -> Optional[str]:
        """Generate chat response from current state/context"""
        message = state['current_message'].lower()
        
        # Simple rule-based responses
        if any(word in message for word in ['hai', 'halo', 'selamat']):
            return "Halo! Selamat datang di Warung AI. Ada yang bisa saya bantu? Anda bisa bertanya tentang produk atau melakukan transaksi."
        
        elif any(word in message for word in ['terima kasih', 'makasih']):
            return "Sama-sama! Senang bisa membantu Anda."
        
        elif 'apa aja' in message or 'ada apa' in message:
            return "Di warung kami ada berbagai macam produk seperti makanan ringan, minuman, dan kebutuhan sehari-hari. Mau cari produk tertentu?"
        
        # If has previous transaction context
        if state.get('transaksi'):
            return f"Anda memiliki transaksi dengan total Rp {state['transaksi']['total_harga_transaksi']:,}. Ada yang ingin ditanyakan?"
        
        return None
    
    def _route_from_chat(self, state: ChatState) -> str:
        """Route from chat handler based on intent"""
        intent = state.get('intent', 'chat')
        
        if intent == 'produk':
            return 'produk'
        elif intent == 'transaksi':
            return 'transaksi'
        else:
            return 'response'
    
    async def _produk_handler_node(self, state: ChatState) -> ChatState:
        """Handle product-related operations (RUD)"""
        message = state['current_message']
        
        try:
            async with MCPServerStdio(params=self.params_produk) as mcp_produk:
                # Extract product info from message
                produk_info = await self._extract_produk_info(message)
                print("produk_info: ", produk_info)
                if produk_info['action'] == 'search':
                    # Search products
                    products = []
                    for item in produk_info['items']:
                        response = await mcp_produk.call_tool(
                            "get_produk_by_name", 
                            arguments={"produk_name": item['name']}
                        )
                        if response.content:
                            produk_data = json.loads(response.content[0].text)
                            products.append({
                                'produk': produk_data,
                                'qty': item.get('qty', 1)
                            })
                    
                    state['produk_terpilih'] = products
                    
                    if products:
                        product_list = "\n".join([
                            f"- {p['produk']['nama_barang']}: Rp {p['produk']['harga']:,} (Stok: {p['produk']['stok']})"
                            for p in products
                        ])
                        state['response'] = f"Produk yang ditemukan:\n{product_list}\n\nApakah Anda ingin membelinya?"
                    else:
                        state['response'] = "Maaf, produk yang Anda cari tidak ditemukan."
                
                elif produk_info['action'] == 'read':
                    # Read specific product
                    response = await mcp_produk.call_tool(
                        "get_all_produk", 
                        arguments={}
                    )
                    if response.content:
                        products = json.loads(response.content[0].text)
                        product_list = "\n".join([
                            f"- {p['nama_barang']}: Rp {p['harga']:,} (Stok: {p['stok']})"
                            for p in products
                        ])
                        state['response'] = f"Daftar produk tersedia:\n{product_list}"
                
        except Exception as e:
            state['response'] = f"Maaf, terjadi kesalahan saat mencari produk: {str(e)}"
        
        return state
    
    async def _extract_produk_info(self, message: str) -> Dict:
        """Extract product information from message"""
        message_lower = message.lower()
        
        # Simple extraction logic
        if any(word in message_lower for word in ['semua', 'list', 'daftar']):
            return {'action': 'read', 'items': []}
        
        # Extract product names and quantities
        # This is simplified - you might want to use LLM for better extraction
        import re
        
        # Look for patterns like "indomie 2", "aqua 3"
        pattern = r'(\w+)\s*(\d+)?'
        matches = re.findall(pattern, message_lower)
        
        items = []
        for match in matches:
            if match[0] not in ['mau', 'beli', 'saya', 'dan']:
                items.append({
                    'name': match[0],
                    'qty': int(match[1]) if match[1] else 1
                })
        
        return {'action': 'search', 'items': items}
    
    async def _transaksi_handler_node(self, state: ChatState) -> ChatState:
        """Handle transaction operations (CRUD)"""
        message = state['current_message']
        
        try:
            async with MCPServerStdio(params=self.params_transaction) as mcp_transaksi:
                transaksi_info = await self._extract_transaksi_info(message)
                
                if transaksi_info['action'] == 'create':
                    # Create transaction from selected products
                    if state.get('produk_terpilih'):
                        result = await self._create_transaction(state, mcp_transaksi)
                        state.update(result)
                        
                        total = state['transaksi']['total_harga_transaksi']
                        state['response'] = f"Transaksi berhasil dibuat dengan total Rp {total:,}. Silakan lakukan pembayaran."
                    else:
                        state['response'] = "Silakan pilih produk terlebih dahulu sebelum membuat transaksi."
                
                elif transaksi_info['action'] == 'read':
                    # Read transactions
                    response = await mcp_transaksi.call_tool("get_all_transactions", arguments={})
                    if response.content:
                        transactions = json.loads(response.content[0].text)
                        if transactions:
                            trans_list = "\n".join([
                                f"- ID {t['id']}: Rp {t['total_harga_transaksi']:,} ({t['status']})"
                                for t in transactions[-5:]  # Show last 5
                            ])
                            state['response'] = f"Transaksi terakhir:\n{trans_list}"
                        else:
                            state['response'] = "Belum ada transaksi."
                
                elif transaksi_info['action'] == 'update':
                    # Update transaction status
                    trans_id = transaksi_info.get('id')
                    if trans_id and state.get('transaksi'):
                        await mcp_transaksi.call_tool(
                            "update_transaction_status",
                            arguments={"transaction_id": trans_id, "status": "completed"}
                        )
                        state['response'] = f"Transaksi ID {trans_id} berhasil diselesaikan."
                
                elif transaksi_info['action'] == 'delete':
                    # Delete transaction
                    trans_id = transaksi_info.get('id')
                    if trans_id:
                        await mcp_transaksi.call_tool(
                            "delete_transaction",
                            arguments={"transaction_id": trans_id}
                        )
                        state['response'] = f"Transaksi ID {trans_id} berhasil dihapus."
                
        except Exception as e:
            state['response'] = f"Maaf, terjadi kesalahan pada transaksi: {str(e)}"
        
        return state
    
    async def _extract_transaksi_info(self, message: str) -> Dict:
        """Extract transaction information from message"""
        message_lower = message.lower()
        
        if any(word in message_lower for word in ['buat', 'checkout', 'bayar', 'beli']):
            return {'action': 'create'}
        elif any(word in message_lower for word in ['lihat', 'cek', 'transaksi']):
            return {'action': 'read'}
        elif any(word in message_lower for word in ['selesai', 'complete', 'update']):
            # Extract ID if present
            import re
            id_match = re.search(r'(\d+)', message)
            trans_id = int(id_match.group(1)) if id_match else None
            return {'action': 'update', 'id': trans_id}
        elif any(word in message_lower for word in ['hapus', 'delete', 'cancel']):
            # Extract ID if present
            import re
            id_match = re.search(r'(\d+)', message)
            trans_id = int(id_match.group(1)) if id_match else None
            return {'action': 'delete', 'id': trans_id}
        
        return {'action': 'create'}  # Default
    
    async def _create_transaction(self, state: ChatState, mcp_transaksi) -> Dict:
        """Create transaction and details"""
        # Calculate total
        total = sum(
            item['produk']['harga'] * item['qty'] 
            for item in state['produk_terpilih']
        )
        
        # Create transaction
        transaksi_data = {
            "tanggal_transaksi": str(datetime.datetime.now()),
            "total_harga_transaksi": total,
            "status": "pending",
            "metode_pembayaran": "tunai",
            "catatan": f"Transaksi pembelian {len(state['produk_terpilih'])} jenis barang"
        }
        
        transaction_response = await mcp_transaksi.call_tool(
            "create_transaction", 
            arguments=transaksi_data
        )
        transaction_data = json.loads(transaction_response.content[0].text)
        
        # Create transaction details
        detail_list = []
        for item in state['produk_terpilih']:
            produk = item['produk']
            qty = item['qty']
            detail_data = {
                "transaction_id": transaction_data['id'],
                "produk_id": produk['id'],
                "qty": qty,
                "harga_per_produk": produk['harga'],
                "total_harga_produk": produk['harga'] * qty,
            }
            
            await mcp_transaksi.call_tool(
                "create_detail_transaction", 
                arguments=detail_data
            )
            detail_list.append(detail_data)
        
        return {
            'transaksi': {
                'id': transaction_data['id'],
                **transaksi_data
            },
            'detail_transaksi': detail_list
        }
    
    async def _response_generator_node(self, state: ChatState) -> ChatState:
        """Generate final response"""
        if not state.get('response'):
            state['response'] = "Maaf, saya tidak dapat memahami permintaan Anda. Bisakah Anda mengulanginya?"
        
        # Add to chat history
        state['chat_history'] = state.get('chat_history', [])
        state['chat_history'].append({
            'user': state['current_message'],
            'assistant': state['response']
        })
        
        return state
    
    async def process_message(self, message: str, history) -> str:
        """Process incoming message through the graph"""
        # Initialize state
        initial_state = ChatState(
            messages=[] if not history else [h[0] for h in history],
            current_message=message,
            chat_history=[] if not history else [{'user': h[0], 'assistant': h[1]} for h in history],
            produk_terpilih=None,
            transaksi=None,
            detail_transaksi=[],
            produk_list=[],
            context={},
            intent=None,
            response=""
        )
        
        # Preserve state from previous interactions if available
        if self.current_state:
            initial_state['produk_terpilih'] = self.current_state.get('produk_terpilih')
            initial_state['transaksi'] = self.current_state.get('transaksi')
            initial_state['context'] = self.current_state.get('context', {})
        
        # Run through graph
        result = await self.graph.ainvoke(initial_state)
        
        # Store current state
        self.current_state = result
        
        return result['response']

# Gradio Interface
def create_gradio_interface():
    """Create Gradio ChatInterface"""
    chatbot = WarungAIChatbot()
    
    async def chat_function(message, history):
        return await chatbot.process_message(message, history)
    
    # Wrapper for async function
    def sync_chat_function(message, history):
        return asyncio.run(chat_function(message, history))
    
    # Create interface
    interface = gr.ChatInterface(
        fn=sync_chat_function,
        title="🏪 Warung AI Chatbot",
        description="Chatbot AI untuk warung yang dapat membantu Anda mencari produk dan melakukan transaksi.",
        examples=[
            "Halo, ada produk apa saja?",
            "Saya mau beli Indomie 2 dan Aqua 3",
            "Berapa harga Indomie?",
            "Buat transaksi",
            "Lihat transaksi saya"
        ],
        submit_btn="📤 Kirim"
    )
    
    return interface

# Main execution
if __name__ == "__main__":
    interface = create_gradio_interface()
    interface.launch()


2025-06-07 13:29:18,260 - produk_database - INFO - init_db success - dropped and recreated table
2025-06-07 13:29:18,263 - transaction_database - INFO - init_db called
2025-06-07 13:29:18,292 - transaction_database - INFO - init_db finished
2025-06-07 13:29:18,304 - produk_database - INFO - create_product_in_db success, product_id=1
2025-06-07 13:29:18,317 - produk_database - INFO - create_product_in_db success, product_id=2
2025-06-07 13:29:18,326 - produk_database - INFO - create_product_in_db success, product_id=3
2025-06-07 13:29:18,337 - produk_database - INFO - create_product_in_db success, product_id=4
2025-06-07 13:29:18,346 - produk_database - INFO - create_product_in_db success, product_id=5
2025-06-07 13:29:18,361 - produk_database - INFO - create_product_in_db success, product_id=6
2025-06-07 13:29:18,371 - produk_database - INFO - create_product_in_db success, product_id=7
2025-06-07 13:29:18,381 - produk_database - INFO - create_product_in_db success, product_id=8
2025-06

* Running on local URL:  http://127.0.0.1:7911
* To create a public link, set `share=True` in `launch()`.


2025-06-07 13:29:19,601 - httpx - INFO - HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"


produk_info:  {'action': 'search', 'items': [{'name': 'indomie', 'qty': 2}, {'name': 'aqua', 'qty': 3}]}


In [40]:
from agents.mcp import MCPServerStdio
import datetime
import json
import re
from produk import ProdukService
from langchain_core.messages import HumanMessage

class ChatState(TypedDict):
    messages: List[str]
    produk_terpilih: Optional[List[Produk]]  # <-- list of Produk
    transaksi: Optional[dict]
    detail_transaksi: List[dict]
    produk_list: List[dict]  

params_produk = {"command": "uv", "args": ["run", "app/mcp_sample/produk_server.py"]}
params_transaction = {"command": "uv", "args": ["run", "app/mcp_sample/transaction_server.py"]}

async def extract_nama_produk_n_qty(state: ChatState):
    message = state['messages'][-1]

    prompt = f"""
    Dari kalimat berikut, ekstrak semua produk dan jumlah (quantity) yang ingin dibeli user.

    Kalimat: "{message}"

    Balas dalam format JSON berupa array seperti ini:
    [
      {{"produk_name": "nama produk", "qty": jumlah}},
      ...
    ]

    Jawab hanya JSON-nya saja, tanpa penjelasan.
    """

    response = await llm.ainvoke([HumanMessage(content=prompt)])
    print(response)

    try:
        parsed = json.loads(response.content)
        state['produk_list'] = parsed
    except Exception as e:
        print("Gagal parsing JSON dari LLM:", e)
        state['produk_list'] = [{"produk_name": "Indomie", "qty": 1}]
    return state


async def cari_produk_node(state: ChatState, mcp_produk: MCPServerStdio):
    produk_list = []

    for item in state['produk_list']:
        produk_name = item['produk_name']
        response = await mcp_produk.call_tool("get_produk_by_name", arguments={"produk_name": produk_name})
        produk_json_str = response.content[0].text
        produk = ProdukService.from_json_report(produk_json_str)
        produk_list.append({"produk": produk, "qty": item['qty']})

    state['produk_terpilih'] = produk_list
    return state


async def buat_transaksi_node(state: ChatState, mcp_transaksi: MCPServerStdio):
    total = sum(
        item['produk'].harga * item['qty'] for item in state['produk_terpilih']
    )

    transaksi_data = {
        "tanggal_transaksi": str(datetime.datetime.now()),
        "total_harga_transaksi": total,
        "status": "pending",
        "metode_pembayaran": "tunai",
        "catatan": f"Transaksi pembelian {len(state['produk_terpilih'])} barang"
    }

    transaction = await mcp_transaksi.call_tool("create_transaction", arguments=transaksi_data)
    transaction_str = transaction.content[0].text
    transaction = json.loads(transaction_str)

    state['transaksi'] = {
        "id": transaction['id'],
        **transaksi_data
    }
    return state


async def buat_detail_transaksi_node(state: ChatState, mcp_transaksi: MCPServerStdio):
    detail_list = []

    for item in state['produk_terpilih']:
        produk = item['produk']
        qty = item['qty']
        detail_data = {
            "transaction_id": state['transaksi']['id'],
            "produk_id": produk.id,
            "qty": qty,
            "harga_per_produk": produk.harga,
            "total_harga_produk": produk.harga * qty,
        }

        await mcp_transaksi.call_tool("create_detail_transaction", arguments=detail_data)
        detail_list.append(detail_data)

    state['detail_transaksi'] = detail_list
    return state



async def cari_produk_runner(state):
    async with MCPServerStdio(params=params_produk) as mcp_produk:
        return await cari_produk_node(state, mcp_produk)

async def buat_transaksi_runner(state):
    async with MCPServerStdio(params=params_transaction) as mcp_transaksi:
        return await buat_transaksi_node(state, mcp_transaksi)

async def buat_detail_runner(state):
    async with MCPServerStdio(params=params_transaction) as mcp_transaksi:
        return await buat_detail_transaksi_node(state, mcp_transaksi)
 

In [41]:
import asyncio
from langgraph.graph import StateGraph, END

async def main():
   

    async with MCPServerStdio(params=params_produk) as mcp_produk:
        async with MCPServerStdio(params=params_transaction) as mcp_transaksi:

            builder = StateGraph(ChatState)

            builder.add_node("cari_produk", cari_produk_runner)
            builder.add_node("buat_transaksi", buat_transaksi_runner)
            builder.add_node("buat_detail", buat_detail_runner)
            builder.add_node("extract_nama_produk_n_qty", extract_nama_produk_n_qty)


            builder.set_entry_point("extract_nama_produk_n_qty")
            builder.add_edge("extract_nama_produk_n_qty", "cari_produk")
            builder.add_edge("cari_produk", "buat_transaksi")
            builder.add_edge("buat_transaksi", "buat_detail")
            builder.add_edge("buat_detail", END)

            graph = builder.compile()

            from IPython.display import Image, display
            mermaid_code = graph.get_graph().draw_mermaid()
            print(mermaid_code)

            # Test input
            input_state = ChatState(messages=["Saya mau beli Indomie 2 dan aqua 2"], produk_terpilih=None, transaksi=None, detail_transaksi=[])

            result = await graph.ainvoke(input_state)

            print("HASIL AKHIR:")
            from pprint import pprint

            pprint(result)

await main()


---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	cari_produk(cari_produk)
	buat_transaksi(buat_transaksi)
	buat_detail(buat_detail)
	extract_nama_produk_n_qty(extract_nama_produk_n_qty)
	__end__([<p>__end__</p>]):::last
	__start__ --> extract_nama_produk_n_qty;
	buat_transaksi --> buat_detail;
	cari_produk --> buat_transaksi;
	extract_nama_produk_n_qty --> cari_produk;
	buat_detail --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



2025-06-06 14:27:44,571 - httpx - INFO - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


content='[\n  {"produk_name": "Indomie", "qty": 2},\n  {"produk_name": "aqua", "qty": 2}\n]' additional_kwargs={} response_metadata={'model': 'llama3-groq-tool-use:latest', 'created_at': '2025-06-06T07:27:54.528980592Z', 'done': True, 'done_reason': 'stop', 'total_duration': 46020024722, 'load_duration': 29283782296, 'prompt_eval_count': 104, 'prompt_eval_duration': 6086285797, 'eval_count': 34, 'eval_duration': 10617699160, 'model_name': 'llama3-groq-tool-use:latest'} id='run--b58d7192-06fc-43a9-9176-3ea475d4162d-0' usage_metadata={'input_tokens': 104, 'output_tokens': 34, 'total_tokens': 138}


2025-06-06 14:27:57,147 - produk - INFO - from_json_report called with json_str={"id":1,"nama_barang":"Indomie Goreng","harga":3000,"lokasi":"Rak Mie Instan","deskripsi_suara_lokasi":"Ada di rak tengah, bagian mie instan.","path_qris":"/qris/indomie_goreng.png","stok":50}
2025-06-06 14:27:57,150 - produk - INFO - from_json_report success: id=1 nama_barang='Indomie Goreng' harga=3000 lokasi='Rak Mie Instan' deskripsi_suara_lokasi='Ada di rak tengah, bagian mie instan.' path_qris='/qris/indomie_goreng.png' stok=50
2025-06-06 14:27:57,160 - produk - INFO - from_json_report called with json_str={"id":2,"nama_barang":"Aqua Botol 600ml","harga":3500,"lokasi":"Kulkas Minuman","deskripsi_suara_lokasi":"Di dalam kulkas minuman, sebelah kanan.","path_qris":"/qris/aqua_600ml.png","stok":100}
2025-06-06 14:27:57,164 - produk - INFO - from_json_report success: id=2 nama_barang='Aqua Botol 600ml' harga=3500 lokasi='Kulkas Minuman' deskripsi_suara_lokasi='Di dalam kulkas minuman, sebelah kanan.' path

HASIL AKHIR:
{'detail_transaksi': [{'harga_per_produk': 3000,
                       'produk_id': 1,
                       'qty': 2,
                       'total_harga_produk': 6000,
                       'transaction_id': 1},
                      {'harga_per_produk': 3500,
                       'produk_id': 2,
                       'qty': 2,
                       'total_harga_produk': 7000,
                       'transaction_id': 1}],
 'messages': ['Saya mau beli Indomie 2 dan aqua 2'],
 'produk_list': [{'produk_name': 'Indomie', 'qty': 2},
                 {'produk_name': 'aqua', 'qty': 2}],
 'produk_terpilih': [{'produk': Produk(id=1, nama_barang='Indomie Goreng', harga=3000, lokasi='Rak Mie Instan', deskripsi_suara_lokasi='Ada di rak tengah, bagian mie instan.', path_qris='/qris/indomie_goreng.png', stok=50),
                      'qty': 2},
                     {'produk': Produk(id=2, nama_barang='Aqua Botol 600ml', harga=3500, lokasi='Kulkas Minuman', deskripsi_suara_loka

In [124]:
import gradio as gr
import asyncio
import json
import datetime
from typing import List, Optional, Dict, Any, TypedDict
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import StateGraph, END, START
from agents.mcp import MCPServerStdio
from produk import ProdukService

# Reset database (assuming this function exists)
reset_db()

# LLM Configuration
llm = ChatOllama(
    model="qwen2.5",
    base_url="http://localhost:11434",
    temperature=0.5,
    top_p=0.8,
    seed=1337
)

# Enhanced State Management
class ChatState(TypedDict):
    messages: List[str]
    current_intent: str
    context: Dict[str, Any]
    produk_terpilih: Optional[List[Dict]]
    transaksi: Optional[Dict]
    detail_transaksi: List[Dict]
    conversation_history: List[Dict]
    user_session: Dict[str, Any]
    cart_total: float
    pending_confirmation: Optional[Dict]
    payment_method: Optional[str]
    customer_name: Optional[str]

# MCP Server Parameters
params_produk = {"command": "uv", "args": ["run", "app/mcp_sample/produk_server.py"]}
params_transaction = {"command": "uv", "args": ["run", "app/mcp_sample/transaction_server.py"]}

class WarungAIBot:
    def __init__(self):
        print("--- Running function: WarungAIBot.__init__ ---")
        self.graph = None
        self.warung_info = {
            "jam_buka": "07:00 - 22:00",
            "layanan": ["snack", "minuman", "kebutuhan sehari-hari"],
            "payment_methods": ["cash"],
            "location": "Warung AI"
        }
        self.current_state = ChatState(
            messages=[],
            current_intent="greeting",
            context={},
            produk_terpilih=[],
            transaksi=None,
            detail_transaksi=[],
            conversation_history=[],
            user_session={"status": "active", "cart": []},
            cart_total=0.0,
            pending_confirmation=None,
            payment_method=None,
            customer_name=None
        )
        self.build_graph()
        print("--- Returning from function: WarungAIBot.__init__ ---")

    def build_graph(self):
        print("--- Running function: WarungAIBot.build_graph ---")
        """Build the enhanced conversation graph"""
        builder = StateGraph(ChatState)
        
        # Add nodes
        builder.add_node("intent_classifier", self.classify_intent)
        builder.add_node("chat_handler", self.handle_chat)
        builder.add_node("produk_handler", self.handle_produk)
        builder.add_node("transaksi_handler", self.handle_transaksi)
        builder.add_node("cart_handler", self.handle_cart)
        builder.add_node("confirmation_handler", self.handle_confirmation)
        builder.add_node("payment_handler", self.handle_payment)
        builder.add_node("info_handler", self.handle_info)
        builder.add_node("response_generator", self.generate_response)
        
        # Set entry point
        builder.set_entry_point("intent_classifier")
        
        # Add conditional edges
        builder.add_conditional_edges(
            "intent_classifier",
            self.route_intent,
            {
                "chat": "chat_handler",
                "produk": "produk_handler", 
                "transaksi": "transaksi_handler",
                "cart": "cart_handler",
                "confirmation": "confirmation_handler",
                "payment": "payment_handler",
                "info": "info_handler",
                "response": "response_generator"
            }
        )
        
        # All handlers lead to response generator
        builder.add_edge("produk_handler", "response_generator")
        builder.add_edge("transaksi_handler", "response_generator")
        builder.add_edge("cart_handler", "response_generator")
        builder.add_edge("confirmation_handler", "response_generator")
        builder.add_edge("payment_handler", "response_generator")
        builder.add_edge("info_handler", "response_generator")
        
        builder.add_edge("response_generator", END)
        self.graph = builder.compile()

        mermaid_code = self.graph.get_graph().draw_mermaid()

        print("--- Mermaid Graph Code ---")

        print(mermaid_code)

        print("--- Returning from function: WarungAIBot.build_graph ---")

    async def classify_intent(self, state: ChatState) -> ChatState:
        print("--- Running function: WarungAIBot.classify_intent ---")
        """Enhanced intent classification"""
        last_message = state['messages'][-1] if state['messages'] else ""
        
        # Check if there's pending confirmation
        if state.get('pending_confirmation'):
            state['current_intent'] = "confirmation"
            return state
        
        intent_prompt = f"""
        Analyze the following user message and determine the intent.

        Previous context: {state.get('context', {})}
        User message: "{last_message}"
        Cart status: {len(state['user_session']['cart'])} items

        Classify into one of the following categories:

        - "chat": includes greetings, small talk, thank-you notes, and farewells. The goal is to make the interaction feel more human and friendly.
        - "produk": this intent is used when users want information about items being sold. Users might ask questions about the products.
        - "cart": for adding/removing items to/from the cart, viewing the cart, or modifying item quantities.
        - "transaksi": for checkout or purchasing, payment, or transaction processing.
        - "payment": for payment confirmation, transfers, or proof of payment.
        - "info": for asking about opening hours, services, or the store's location.
        - "confirmation": for yes/no confirmations, correct/incorrect, continue/stop.

        Respond with the category name only.
        """
        
        response = await llm.ainvoke([HumanMessage(content=intent_prompt)])
        intent = response.content.strip().lower()
        # print("intent: ", intent)
        # Fallback to chat if intent unclear
        valid_intents = ["chat", "produk", "cart", "transaksi", "payment", "info", "confirmation"]
        if intent not in valid_intents:
            intent = "chat"
            
        state['current_intent'] = intent
        print(f"--- Returning from function: WarungAIBot.classify_intent with intent: {intent} ---")
        return state

    def route_intent(self, state: ChatState) -> str:
        print("--- Running function: WarungAIBot.route_intent ---")
        intent = state['current_intent']
        print(f"--- Returning from function: WarungAIBot.route_intent with route: {intent} ---")
        return intent

    async def handle_chat(self, state: ChatState) -> ChatState:
        print("--- Running function: WarungAIBot.handle_chat ---")
        """Handle general chat interactions"""
        last_message = state['messages'][-1]
        
        chat_context = {
            "type": "general_chat",
            "user_message": last_message,
            "session_info": state['user_session'],
            "greeting": "Selamat datang di Warung AI! Ada yang bisa saya bantu?"
        }
        
        state['context'].update(chat_context)
        print("--- Returning from function: WarungAIBot.handle_chat ---")
        return state

    async def handle_produk(self, state: ChatState) -> ChatState:
        print("--- Running function: WarungAIBot.handle_produk ---")
        """Enhanced product handling with variants and details"""
        last_message = state['messages'][-1]
        
        extract_prompt = f"""
        Here's the English translation of your prompt:

        From the user message: "{last_message}"

        Extract the following information in JSON format:

        {{
            "action": "choose one from: search or list",
            "produk_name": "extract the name of the item, product, or brand mentioned. If the message only mentions a category (e.g., 'snacks', 'drinks') or something too general, or if nothing is mentioned, then use None.",
            "quantity": "extract the quantity if mentioned, otherwise use 0",
            "variant_requested": "extract variant such as size, flavor, color if mentioned, if none or ambiguous then use None"
        }}
        

        Make sure the JSON is complete and must include action, produk_name, quantity, and variant_requested — DO NOT omit any of them.

        Respond with the JSON only.
        """
        
        try:
            response = await llm.ainvoke([HumanMessage(content=extract_prompt)])
            produk_info = json.loads(response.content)
        except:
            produk_info = {"action": "list", "produk_name": None, "quantity": 0, "variant_requested": None}
        
        print(f"produk_info: {produk_info}")
        # Get product data via MCP
        async with MCPServerStdio(params=params_produk) as mcp_produk:
            if produk_info["action"] == "search" and produk_info["produk_name"]:
                response = await mcp_produk.call_tool(
                    "get_produk_by_name", 
                    arguments={"produk_name": produk_info["produk_name"]}
                )
                
                semua_produk = []
                for item in response.content:
                    semua_produk.append(json.loads(item.text))
                    
                produk_data = semua_produk
                
            else:
                response = await mcp_produk.call_tool("list_all_produk", arguments={})
                
                semua_produk = []
                for item in response.content:
                    semua_produk.append(json.loads(item.text))
                    
                produk_data = semua_produk

        state['context'].update({
            "type": "produk_inquiry",
            "action": produk_info["action"],
            "produk_data": produk_data,
            "requested_product": produk_info["produk_name"],
            "quantity": produk_info["quantity"],
            "variant_requested": produk_info["variant_requested"]
        })
        
        print("--- Returning from function: WarungAIBot.handle_produk ---")
        return state

    async def handle_cart(self, state: ChatState) -> ChatState:
        print("--- Running function: WarungAIBot.handle_cart ---")
        """Enhanced cart operations with quantity modification"""
        last_message = state['messages'][-1]
        
        cart_prompt = f"""
            From the message: "{last_message}"  
            Current cart context: {state['user_session']['cart']}

            Determine the action for the cart in JSON format:
            {
                "action": "add|remove|view|clear|modify",
                "produk_name": "product name or null",
                "quantity": "quantity or 1",
                "modification_type": "increase|decrease|set|null"
            }

            Examples:
            - "Add 2 Indomie" -> {"action": "add", "produk_name": "Indomie", "quantity": 2, "modification_type": null}
            - "Change Chitato to just 1" -> {"action": "modify", "produk_name": "Chitato", "quantity": 1, "modification_type": "set"}
            - "Remove Aqua" -> {"action": "remove", "produk_name": "Aqua", "quantity": 1, "modification_type": null}

            Respond with JSON only.
        """

        
        try:
            response = await llm.ainvoke([HumanMessage(content=cart_prompt)])
            cart_info = json.loads(response.content)
        except:
            cart_info = {"action": "view", "produk_name": None, "quantity": 1, "modification_type": None}

        # Process cart action
        if cart_info["action"] == "add" and cart_info["produk_name"]:
            # Check if product exists in database first
            async with MCPServerStdio(params=params_produk) as mcp_produk:
                response = await mcp_produk.call_tool(
                    "get_produk_by_name", 
                    arguments={"produk_name": cart_info["produk_name"]}
                )
                
                produk_found = []
                for item in response.content:
                    produk_found.append(json.loads(item.text))
                
                if produk_found:
                    produk = produk_found[0]  # Take first match
                    existing_item = next(
                        (item for item in state['user_session']['cart'] 
                         if item['name'].lower() == cart_info["produk_name"].lower()), None
                    )
                    
                    if existing_item:
                        existing_item['quantity'] += cart_info["quantity"]
                    else:
                        state['user_session']['cart'].append({
                            'name': produk['nama_produk'],
                            'price': produk['harga_produk'],
                            'quantity': cart_info["quantity"]
                        })
                else:
                    state['context'].update({
                        "type": "cart_error",
                        "message": f"Produk {cart_info['produk_name']} tidak ditemukan"
                    })
                    return state

        elif cart_info["action"] == "modify" and cart_info["produk_name"]:
            existing_item = next(
                (item for item in state['user_session']['cart'] 
                 if item['name'].lower() == cart_info["produk_name"].lower()), None
            )
            
            if existing_item:
                if cart_info["modification_type"] == "set":
                    existing_item['quantity'] = cart_info["quantity"]
                elif cart_info["modification_type"] == "increase":
                    existing_item['quantity'] += cart_info["quantity"]
                elif cart_info["modification_type"] == "decrease":
                    existing_item['quantity'] = max(0, existing_item['quantity'] - cart_info["quantity"])
                    if existing_item['quantity'] == 0:
                        state['user_session']['cart'].remove(existing_item)

        elif cart_info["action"] == "remove" and cart_info["produk_name"]:
            state['user_session']['cart'] = [
                item for item in state['user_session']['cart']
                if item['name'].lower() != cart_info["produk_name"].lower()
            ]

        elif cart_info["action"] == "clear":
            state['user_session']['cart'] = []

        # Calculate total
        total = sum(item['price'] * item['quantity'] for item in state['user_session']['cart'])
        state['cart_total'] = total

        state['context'].update({
            "type": "cart_operation",
            "action": cart_info["action"],
            "cart_contents": state['user_session']['cart'],
            "cart_total": total,
            "item_count": len(state['user_session']['cart'])
        })
        
        print("--- Returning from function: WarungAIBot.handle_cart ---")
        return state

    async def handle_confirmation(self, state: ChatState) -> ChatState:
        print("--- Running function: WarungAIBot.handle_confirmation ---")
        """Handle confirmation responses"""
        last_message = state['messages'][-1].lower()
        
        # Simple confirmation detection
        positive_responses = ['ya', 'benar', 'iya', 'ok', 'oke', 'lanjut', 'setuju', 'yes']
        negative_responses = ['tidak', 'nggak', 'enggak', 'batal', 'no']
        
        is_positive = any(word in last_message for word in positive_responses)
        is_negative = any(word in last_message for word in negative_responses)
        
        pending = state.get('pending_confirmation')
        if pending:
            if is_positive:
                if pending['type'] == 'cart_total':
                    # Proceed to checkout
                    state['context'].update({
                        "type": "proceed_to_checkout",
                        "confirmed": True,
                        "cart_total": state['cart_total']
                    })
                elif pending['type'] == 'transaction':
                    # Process payment
                    state['context'].update({
                        "type": "process_payment",
                        "confirmed": True
                    })
            elif is_negative:
                state['context'].update({
                    "type": "confirmation_declined",
                    "message": "Baik, ada yang ingin diubah?"
                })
            
            # Clear pending confirmation
            state['pending_confirmation'] = None
        
        print("--- Returning from function: WarungAIBot.handle_confirmation ---")
        return state

    async def handle_payment(self, state: ChatState) -> ChatState:
        print("--- Running function: WarungAIBot.handle_payment ---")
        """Handle payment processing"""
        last_message = state['messages'][-1]
        
        # Extract customer name if provided
        name_prompt = f"""
        From the message: "{last_message}"

        Is there a customer name mentioned?
        If yes, extract the name.
        If not, respond with "null".

        Respond with the name only or "null".
        """
        
        try:
            response = await llm.ainvoke([HumanMessage(content=name_prompt)])
            customer_name = response.content.strip()
            if customer_name.lower() != "null":
                state['customer_name'] = customer_name
        except:
            pass

        state['context'].update({
            "type": "payment_processing",
            "payment_method": "cash",  # Only cash for now
            "customer_name": state.get('customer_name'),
            "cart_total": state['cart_total']
        })
        
        print("--- Returning from function: WarungAIBot.handle_payment ---")
        return state

    async def handle_info(self, state: ChatState) -> ChatState:
        print("--- Running function: WarungAIBot.handle_info ---")
        """Handle warung information requests"""
        last_message = state['messages'][-1]
        
        info_context = {
            "type": "warung_info",
            "jam_buka": self.warung_info["jam_buka"],
            "layanan": self.warung_info["layanan"],
            "payment_methods": self.warung_info["payment_methods"],
            "user_question": last_message
        }
        
        state['context'].update(info_context)
        print("--- Returning from function: WarungAIBot.handle_info ---")
        return state

    async def handle_transaksi(self, state: ChatState) -> ChatState:
        print("--- Running function: WarungAIBot.handle_transaksi ---")
        """Enhanced transaction processing"""
        last_message = state['messages'][-1]
        
        if not state['user_session']['cart']:
            state['context'].update({
                "type": "transaction_error",
                "message": "Keranjang kosong, silakan pilih produk terlebih dahulu."
            })
            return state

        # Check if this is initial checkout request
        if not state.get('pending_confirmation'):
            # Set pending confirmation for cart total
            state['pending_confirmation'] = {
                "type": "cart_total",
                "cart_contents": state['user_session']['cart'],
                "total": state['cart_total']
            }
            
            state['context'].update({
                "type": "checkout_confirmation",
                "cart_contents": state['user_session']['cart'],
                "cart_total": state['cart_total'],
                "needs_confirmation": True
            })
            return state

        # If confirmed, process transaction
        try:
            async with MCPServerStdio(params=params_transaction) as mcp_transaksi:
                transaksi_data = {
                    "tanggal_transaksi": str(datetime.datetime.now()),
                    "total_harga_transaksi": state['cart_total'],
                    "status": "completed",
                    "metode_pembayaran": "cash",
                    "catatan": f"Transaksi {len(state['user_session']['cart'])} items"
                }
                
                transaction = await mcp_transaksi.call_tool(
                    "create_transaction", 
                    arguments=transaksi_data
                )
                
                state['transaksi'] = json.loads(transaction.content[0].text)
                
                # Create transaction details
                for item in state['user_session']['cart']:
                    detail_data = {
                        "id_transaksi": state['transaksi']['id'],
                        "nama_produk": item['name'],
                        "jumlah_produk": item['quantity'],
                        "harga_satuan": item['price'],
                        "subtotal": item['price'] * item['quantity']
                    }
                    
                    detail = await mcp_transaksi.call_tool(
                        "create_transaction_detail",
                        arguments=detail_data
                    )
                    state['detail_transaksi'].append(json.loads(detail.content[0].text))
                
                # Clear cart after successful transaction
                state['user_session']['cart'] = []
                state['cart_total'] = 0.0
                
                state['context'].update({
                    "type": "transaction_success",
                    "transaction_id": state['transaksi']['id'],
                    "customer_name": state.get('customer_name'),
                    "total": state['transaksi']['total_harga_transaksi']
                })
                
        except Exception as e:
            state['context'].update({
                "type": "transaction_error",
                "message": f"Gagal memproses transaksi: {str(e)}"
            })
        
        print("--- Returning from function: WarungAIBot.handle_transaksi ---")
        return state

    async def generate_response(self, state: ChatState) -> ChatState:
        print("--- Running function: WarungAIBot.generate_response ---")
        """Enhanced response generation with context-aware responses"""
        context = state['context']
        last_message = state['messages'][-1] if state['messages'] else ""
        context_type = context.get('type', 'general')
        
        # Context-specific response generation
                # Context-specific response generation
        if context_type == "general_chat":
            response_prompt = f"""
            You are a friendly AI assistant for a small shop. The user said: "{last_message}"

            Give a warm greeting and ask how you can help.
            Use friendly and natural Indonesian language.
            """
            
        elif context_type == "produk_inquiry":
            produk_data = context.get('produk_data', [])
            action = context.get('action')
            requested_product = context.get('requested_product')
            
            if action == "availability" and produk_data:
                # Show available products with variants
                response_prompt = f"""
                User asked about availability: "{requested_product}"
                
                Found product data:
                {json.dumps(produk_data, indent=2)}
                
                Provide an informative response about product availability.
                If there are variants (e.g., size, flavor), mention the available options.
                Ask for specific preferences from the user if necessary.
                """
            elif not produk_data:
                response_prompt = f"""
                Product "{requested_product}" not found.
                Offer alternatives or ask for clarification from the user.
                """
            else:
                response_prompt = f"""
                Product data:
                {json.dumps(produk_data[:5], indent=2)}  # Limit to 5 items
                
                Display the product list including name, price, and other important details.
                Use an easy-to-read format.
                """
                
        elif context_type == "cart_operation":
            cart_contents = context.get('cart_contents', [])
            cart_total = context.get('cart_total', 0)
            action = context.get('action')
            
            response_prompt = f"""
            Cart operation: {action}
            Current cart contents:
            {json.dumps(cart_contents, indent=2)}
            Total: Rp {cart_total:,.0f}
            
            Confirm the action taken and show the updated cart contents.
            """
            
        elif context_type == "checkout_confirmation":
            cart_contents = context.get('cart_contents', [])
            cart_total = context.get('cart_total', 0)
            
            response_prompt = f"""
            User wants to checkout with the following cart:
            {json.dumps(cart_contents, indent=2)}
            Total: Rp {cart_total:,.0f}
            
            Display a summary of the order and ask for confirmation if it's correct.
            Format it like: "Alright, [item list]. The total is Rp [total]. Is that correct?"
            """
            
        elif context_type == "transaction_success":
            transaction_id = context.get('transaction_id')
            customer_name = context.get('customer_name')
            total = context.get('total', 0)
            
            response_prompt = f"""
            Transaction successful!
            Transaction ID: {transaction_id}
            Customer: {customer_name or 'Customer'}
            Total: Rp {total:,.0f}
            
            Give a thank-you message and confirm that the order will be prepared shortly.
            """
            
        elif context_type == "warung_info":
            jam_buka = context.get('jam_buka')
            layanan = context.get('layanan', [])
            user_question = context.get('user_question')
            
            response_prompt = f"""
            User asked: "{user_question}"
            
            Shop information:
            - Opening hours: {jam_buka}
            - Services: {', '.join(layanan)}
            - Payment: Cash
            
            Respond to the user's question with relevant information.
            """
            
        else:
            response_prompt = f"""
            Context: {json.dumps(context, indent=2)}
            User: "{last_message}"
            
            Berikan respons yang sesuai dengan konteks.
            """

        response = await llm.ainvoke([HumanMessage(content=response_prompt)])
        
        # Add to conversation history
        state['conversation_history'].append({
            "user": last_message,
            "assistant": response.content,
            "context": context.copy(),
            "timestamp": datetime.datetime.now().isoformat()
        })
        
        print("--- Returning from function: WarungAIBot.generate_response ---")
        return state

    async def process_message(self, message: str) -> str:
        print(f"--- Running function: WarungAIBot.process_message with message: '{message}' ---")
        """Process user message through the enhanced graph"""
        # Add message to state
        self.current_state['messages'].append(message)
        
        # Run through graph
        result = await self.graph.ainvoke(self.current_state)
        
        # Update current state
        self.current_state = result
        
        # Return last assistant response
        if result['conversation_history']:
            final_response = result['conversation_history'][-1]['assistant']
            print(f"--- Returning response: '{final_response}' ---")
            return final_response
        else:
            error_message = "Maaf, saya tidak bisa memproses pesan Anda saat ini."
            print(f"--- Returning error: '{error_message}' ---")
            return error_message
        
warung_bot = WarungAIBot()
warung_bot.build_graph()

2025-06-07 12:27:42,134 - produk_database - INFO - init_db success - dropped and recreated table
2025-06-07 12:27:42,136 - transaction_database - INFO - init_db called
2025-06-07 12:27:42,162 - transaction_database - INFO - init_db finished
2025-06-07 12:27:42,172 - produk_database - INFO - create_product_in_db success, product_id=1
2025-06-07 12:27:42,180 - produk_database - INFO - create_product_in_db success, product_id=2
2025-06-07 12:27:42,189 - produk_database - INFO - create_product_in_db success, product_id=3
2025-06-07 12:27:42,199 - produk_database - INFO - create_product_in_db success, product_id=4
2025-06-07 12:27:42,214 - produk_database - INFO - create_product_in_db success, product_id=5
2025-06-07 12:27:42,221 - produk_database - INFO - create_product_in_db success, product_id=6
2025-06-07 12:27:42,231 - produk_database - INFO - create_product_in_db success, product_id=7
2025-06-07 12:27:42,242 - produk_database - INFO - create_product_in_db success, product_id=8
2025-06

--- Running function: WarungAIBot.__init__ ---
--- Running function: WarungAIBot.build_graph ---
--- Mermaid Graph Code ---
---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	intent_classifier(intent_classifier)
	chat_handler(chat_handler)
	produk_handler(produk_handler)
	transaksi_handler(transaksi_handler)
	cart_handler(cart_handler)
	confirmation_handler(confirmation_handler)
	payment_handler(payment_handler)
	info_handler(info_handler)
	response_generator(response_generator)
	__end__([<p>__end__</p>]):::last
	__start__ --> chat_handler;
	cart_handler --> response_generator;
	chat_handler --> intent_classifier;
	confirmation_handler --> response_generator;
	info_handler --> response_generator;
	intent_classifier -. &nbsp;cart&nbsp; .-> cart_handler;
	intent_classifier -. &nbsp;chat&nbsp; .-> chat_handler;
	intent_classifier -. &nbsp;confirmation&nbsp; .-> confirmation_handler;
	intent_classifier -. &nbsp;info&nbsp; .-> info_handler;
	int

In [101]:
# Gradio Interface
warung_bot = WarungAIBot()

async def chat_fn(message, history):
    print(f"--- Running function: chat_fn with message: '{message}' ---")
    """Gradio chat interface function"""
    try:
        response = await warung_bot.process_message(message)
        print("--- Returning from function: chat_fn (success) ---")
        return response
    except Exception as e:
        error_response = f"Maaf, terjadi kesalahan: {str(e)}"
        print(f"--- Returning from function: chat_fn (exception): {error_response} ---")
        return error_response

def create_chat_interface():
    """Membuat Gradio Chat Interface yang lebih sederhana dan modern."""
    
    # Contoh percakapan untuk ditampilkan di UI
    example_conversations = [
        "Halo! Ada produk apa saja?",
        "Cari Indomie dong",
        "Berapa harganya?",
        "Tambah 3 ke keranjang",
        "Keranjang saya ada apa aja?",
        "Checkout yuk!",
        "Hapus Indomie dari keranjang"
    ]

    # Membuat antarmuka dengan gr.ChatInterface
    iface = gr.ChatInterface(
        fn=chat_fn,
        title="🛒 Dynamic Warung AI",
        description="""
        ### Asisten Belanja Cerdas
        Selamat datang! Saya adalah asisten AI yang dapat membantu Anda mencari produk, mengelola keranjang belanja, dan memproses transaksi.
        """,
        examples=example_conversations,
        theme=gr.themes.Soft(),
        chatbot=gr.Chatbot(height=500, placeholder="Mulai percakapan...", bubble_full_width=False),
        textbox=gr.Textbox(placeholder="Ketik pertanyaan atau permintaan Anda...", container=False, scale=7),
    )
    return iface

# Run the application
if __name__ == "__main__":
    print("--- Starting main execution block ---")
    iface = create_chat_interface()
    iface.launch(
        share=False,
        show_error=True
    )

--- Running function: WarungAIBot.__init__ ---
--- Running function: WarungAIBot.build_graph ---
--- Mermaid Graph Code ---
---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	intent_classifier(intent_classifier)
	chat_handler(chat_handler)
	produk_handler(produk_handler)
	transaksi_handler(transaksi_handler)
	cart_handler(cart_handler)
	confirmation_handler(confirmation_handler)
	payment_handler(payment_handler)
	info_handler(info_handler)
	response_generator(response_generator)
	__end__([<p>__end__</p>]):::last
	__start__ --> intent_classifier;
	cart_handler --> response_generator;
	chat_handler --> response_generator;
	confirmation_handler --> response_generator;
	info_handler --> response_generator;
	intent_classifier -. &nbsp;cart&nbsp; .-> cart_handler;
	intent_classifier -. &nbsp;chat&nbsp; .-> chat_handler;
	intent_classifier -. &nbsp;confirmation&nbsp; .-> confirmation_handler;
	intent_classifier -. &nbsp;info&nbsp; .-> info_handler

/tmp/ipykernel_98265/820536448.py:40: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot=gr.Chatbot(height=500, placeholder="Mulai percakapan...", bubble_full_width=False),
/tmp/ipykernel_98265/820536448.py:40: DeprecationWarning: The 'bubble_full_width' parameter is deprecated and will be removed in a future version. This parameter no longer has any effect.
  chatbot=gr.Chatbot(height=500, placeholder="Mulai percakapan...", bubble_full_width=False),
/home/lenov/Documents/warung/.venv/lib/python3.12/site-packages/gradio/chat_interface.py:321: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'tuples', will be used.
  warnings.warn(
2025-06-06 19:26:26,993 - httpx - INFO - HTTP Reque

* Running on local URL:  http://127.0.0.1:7908
* To create a public link, set `share=True` in `launch()`.


2025-06-06 19:26:27,596 - httpx - INFO - HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"


--- Running function: chat_fn with message: 'ada minuman apa saja?' ---
--- Running function: WarungAIBot.process_message with message: 'ada minuman apa saja?' ---
--- Running function: WarungAIBot.classify_intent ---


...

--- Returning from function: WarungAIBot.classify_intent with intent: produk ---
--- Running function: WarungAIBot.route_intent ---
--- Returning from function: WarungAIBot.route_intent with route: produk ---
--- Running function: WarungAIBot.handle_produk ---


2025-06-06 19:28:54,662 - httpx - INFO - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


produk_info: {'action': 'list', 'produk_name': 'drinks', 'quantity': 0, 'variant_requested': None}
--- Returning from function: WarungAIBot.handle_produk ---
--- Running function: WarungAIBot.generate_response ---


2025-06-06 19:29:06,045 - httpx - INFO - HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


--- Returning from function: WarungAIBot.generate_response ---
--- Returning response: 'Here is the product list:

1. **Indomie Goreng**
   - Price: IDR 3,000
   - Location: Rak Mie Instan
   - Description: Ada di rak tengah, bagian mie instan.

2. **Aqua Botol 600ml**
   - Price: IDR 3,500
   - Location: Kulkas Minuman
   - Description: Di dalam kulkas minuman, sebelah kanan.

3. **Chitato Sapi Panggang**
   - Price: IDR 10,000
   - Location: Rak Snack
   - Description: Rak snack, di bagian atas.

4. **Susu Ultra Coklat 250ml**
   - Price: IDR 6,000
   - Location: Kulkas Susu
   - Description: Kulkas khusus produk susu, di baris kedua dari atas.

5. **Roti Tawar Sari Roti**
   - Price: IDR 15,000
   - Location: Rak Roti
   - Description: Area roti, dekat kasir.

Is there anything else you need help with?' ---
--- Returning from function: chat_fn (success) ---
